In [1]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical


In [ ]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
data = pd.read_csv("./sample_data/Sentiment.csv")

# Select only the necessary columns 'text' and 'sentiment'
mask = data.columns.isin(['text', 'sentiment'])
data = data.loc[:, mask]


In [ ]:
# Keeping only the necessary columns
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [ ]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets


In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix

In [ ]:
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons

In [ ]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())

In [ ]:
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split


In [ ]:
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 62s - loss: 0.8254 - accuracy: 0.6445 - 62s/epoch - 212ms/step
144/144 - 2s - loss: 0.7541 - accuracy: 0.6737 - 2s/epoch - 11ms/step
0.7541212439537048
0.6736566424369812


In [ ]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [ ]:
model.save('sentimentAnalysis.h5') #Saving the model

In [ ]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [ ]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [ ]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")

1/1 - 0s - 277ms/epoch - 277ms/step
[0.5626313  0.1534134  0.28395534]
Neutral


Apply GridSearchCV on the source code provided in the class

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

<ipython-input-15-6c99b49150f4>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 108s - loss: 0.8287 - accuracy: 0.6468 - 108s/epoch - 146ms/step
186/186 - 2s - loss: 0.7715 - accuracy: 0.6622 - 2s/epoch - 12ms/step


744/744 - 110s - loss: 0.8246 - accuracy: 0.6461 - 110s/epoch - 148ms/step
186/186 - 2s - loss: 0.7600 - accuracy: 0.6772 - 2s/epoch - 12ms/step


744/744 - 110s - loss: 0.8281 - accuracy: 0.6481 - 110s/epoch - 147ms/step
186/186 - 2s - loss: 0.7672 - accuracy: 0.6799 - 2s/epoch - 13ms/step


744/744 - 110s - loss: 0.8224 - accuracy: 0.6460 - 110s/epoch - 148ms/step
186/186 - 3s - loss: 0.7377 - accuracy: 0.6851 - 3s/epoch - 16ms/step


...

Epoch 1/2
186/186 - 37s - loss: 0.8388 - accuracy: 0.6402 - 37s/epoch - 197ms/step
Epoch 2/2
186/186 - 25s - loss: 0.6917 - accuracy: 0.7016 - 25s/epoch - 137ms/step
47/47 - 1s - loss: 0.7352 - accuracy: 0.6799 - 751ms/epoch - 16ms/step


Epoch 1/2
186/186 - 35s - loss: 0.8423 - accuracy: 0.6337 - 35s/epoch - 190ms/step
Epoch 2/2
186/186 - 25s - loss: 0.6904 - accuracy: 0.7072 - 25s/epoch - 136ms/step
47/47 - 1s - loss: 0.7554 - accuracy: 0.6842 - 721ms/epoch - 15ms/step


Epoch 1/2
186/186 - 36s - loss: 0.8499 - accuracy: 0.6301 - 36s/epoch - 196ms/step
Epoch 2/2
186/186 - 25s - loss: 0.6926 - accuracy: 0.7018 - 25s/epoch - 134ms/step
47/47 - 1s - loss: 0.7682 - accuracy: 0.6760 - 760ms/epoch - 16ms/step


Epoch 1/2
186/186 - 35s - loss: 0.8465 - accuracy: 0.6319 - 35s/epoch - 187ms/step
Epoch 2/2
186/186 - 24s - loss: 0.6815 - accuracy: 0.7084 - 24s/epoch - 131ms/step
47/47 - 1s - loss: 0.7744 - accuracy: 0.6625 - 1s/epoch - 23ms/step


Epoch 1/2
465/465 - 69s - loss: 0.8144 - accuracy: 0.6503 - 69s/epoch - 149ms/step
Epoch 2/2
465/465 - 59s - loss: 0.6750 - accuracy: 0.7143 - 59s/epoch - 126ms/step
Best: 0.680081 using {'batch_size': 20, 'epochs': 2}
